# Using External Tables from BigQuery

BigQuery has the ability to query data directly from Google Cloud Storage (a feature called Federated Data Sources). This can be useful when querying small amounts of data that you may not want to load into a BigQuery table. It is not recommended for large queries, because BigQuery billing is based on the amount of data read to process a query. BigQuery can very efficiently query subsets of tables in its own store as these are stored in columnar format, so the unused columns are not read and don't add any cost. Data stored in GCS is typically going to be in a possibly compressed CSV file and the entire file will need to be read. So this is a useful feature but should be used judiciously. 

In this notebook we will show you how to download some data from a source on the Internet, put it in GCS, and then query it directly.

## Getting the Data and Loading into GCS

For this sample we want to use some external data in a CSV, load it into GCS and query it. The data we will use is the Seattle bike station data from the [Pronto 2015 Data Challenge dataset](https://www.prontocycleshare.com/datachallenge).


In [1]:
import gcp
import gcp.bigquery as bq
import gcp.storage as gs

In [2]:
import urllib2

data_source = "https://storage.googleapis.com/cloud-datalab-samples/udfsample/2015_station_data.csv"

f = urllib2.urlopen(data_source)
data = f.read()
f.close()

print 'Read %d bytes' % len(data)

Read 4230 bytes


In [3]:
# Get a bucket in the current project
project = gcp.Context.default().project_id
sample_bucket_name = project + '-station_data'

# Create and write to the GCS item
sample_bucket = gs.Bucket(sample_bucket_name)
sample_bucket.create()
sample_item = sample_bucket.item('station_data.csv')
sample_item.write_to(data, 'text/plain')

## Creating a FederatedTable Object

Now we need to create a special FederatedTable object that refers to the data, and that can in turn be used as a table in our BigQuery queries. We need to provide a schema for BigQuery to be able to use the data. The CSV file has a header row that we want to skip; we will use the `CSVOptions` for this.


In [4]:
options = bq.CSVOptions(skip_leading_rows=1)  # Skip the header row

schema = bq.Schema([
  {'name': 'id', 'type': 'INTEGER'},         # row ID
  {'name': 'name', 'type': 'STRING'},        # friendly name
  {'name': 'terminal', 'type': 'STRING'},    # terminal ID
  {'name': 'lat', 'type': 'FLOAT'},          # latitude
  {'name': 'long', 'type': 'FLOAT'},         # longitude
  {'name': 'dockcount', 'type': 'INTEGER'},  # bike capacity
  {'name': 'online', 'type': 'STRING'}       # date station opened
])

drivedata = bq.FederatedTable.from_storage(sample_item.uri, # The gs:// URL of the file
                                           csv_options=options,
                                           schema=schema,
                                           max_bad_records=10)


## Querying the Table

Now let's verify that we can access the data. We will run a simple query to show the first 5 rows. Note how we specify the federated table by using just a name in the query, and then passing the table in using a data_sources dictionary parameter.

In [5]:
bq.Query('SELECT * FROM data LIMIT 5', data_sources={'data': drivedata}).results()

id,name,terminal,lat,long,dockcount,online
1,3rd Ave & Broad St,BT-01,47.6184196472,-122.350967407,18,10/13/2014
2,2nd Ave & Vine St,BT-03,47.6158294678,-122.348564148,16,10/13/2014
3,6th Ave & Blanchard St,BT-04,47.6160926819,-122.3411026,16,10/13/2014
4,2nd Ave & Blanchard St,BT-05,47.6131095886,-122.344207764,14,10/13/2014
5,2nd Ave & Pine St,CBD-13,47.6101837158,-122.339637756,18,10/13/2014


Finally, let's clean up.

In [6]:
sample_item.delete()
sample_bucket.delete()